# DEEP AUTOENCODER-- meaning have more than one autoencoder and more than one deep autoencoder

In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
import seaborn as sns

C:\Users\obaid\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\obaid\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\obaid\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\obaid\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passi

In [3]:
import keras 
print(keras.__version__)

2.2.4


Using TensorFlow backend.


In [4]:
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint , TensorBoard
from keras import regularizers

In [5]:
df = pd.read_csv("creditcard.csv")

In [6]:
df.head(3)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0


In [7]:
df.shape

(284807, 31)

# PreProcessing

In [8]:
from sklearn.preprocessing import StandardScaler
data= df.drop(['Time'],axis=1)   ### drop the time column
data['Amount']=StandardScaler().fit_transform(data['Amount'].values.reshape(-1,1))

# Split Dataset 

In [9]:
X_train, X_test = train_test_split(data,test_size=0.25,random_state=123)
X_train = X_train[X_train.Class == 0]  ### only class 0 should be in X_train
X_train = X_train.drop(['Class'],axis=1)
y_test = X_test['Class']
X_test = X_test.drop(['Class'],axis=1)
X_train = X_train.values
X_test = X_test.values

In [11]:
type(X_train)

numpy.ndarray

In [12]:
type(X_test)

numpy.ndarray

In [13]:
X_train.shape

(213260, 29)

In [14]:
X_test.shape

(71202, 29)

#  Autoencoder uses 4 fully connected layers with 14,7,7 and 29 neurons respectively. 

# First two layers are for encoder and last two go for decoder

# L1 regularization

In [15]:
input_dim = X_train.shape[1]
encoding_dim = 14 

In [16]:
input_layer = Input(shape=(input_dim,  ))   ### this will be 29 as there are 29 features coming in as numpy array

# Define Encoder

In [17]:
encoder= Dense(encoding_dim, activation="tanh",   
              activity_regularizer=regularizers.l1(10e-5))(input_layer)  ### 29 going to 14 

In [18]:
encoder = Dense(int(encoding_dim/2),activation='relu')(encoder)    ### above layer going to multiply with this

# Define Decoder

In [19]:
decoder = Dense(int(encoding_dim/2),activation='tanh')(encoder)
decoder = Dense(input_dim,activation='relu')(decoder)

# AUTOENCODER

In [20]:
autoencoder=Model(inputs=input_layer , outputs=decoder)

# Run autoencoder for 100 epochs with a batch size of 32

In [21]:
nb_epoch = 80
batch_size= 32

In [22]:
autoencoder.compile(optimizer='adam',
                   loss='mean_squared_error',
                   metrics=['accuracy'])